## Accuracy

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

In [2]:
from catalyst.dl.callbacks import Callback, AccuracyCallback
from catalyst.dl.experiments import SupervisedRunner

from collections import OrderedDict, defaultdict

import torch
import numpy as np

In [3]:
loaders = OrderedDict()

In [4]:
import sys
sys.path.append('../')
import dataloaders

import torchvision.datasets as datasets
from torchvision import transforms

# DATA_ROOT = "/workspace/raid/data/datasets"
DATA_ROOT = "/gpfs/gpfs0/e.ponomarev/"
dataset_name = 'imagenet'

bs = 128
num_workers = 32

if dataset_name == 'cifar10':
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    DATA_PATH = "{}/cifar10".format(DATA_ROOT)

    loaders["valid"] = torch.utils.data.DataLoader(
        datasets.CIFAR10(root=DATA_PATH, train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ]),  download = True),
        batch_size=bs, shuffle=False,
        num_workers=num_workers, pin_memory=True)
elif dataset_name == 'imagenet':
    loaders["valid"] = dataloaders.get_loader(batch_size=bs,
                                        data_name = 'imagenet',
                                        data_root = DATA_ROOT,
                                        num_workers = num_workers, 
                                        pin_memory = True)['val']
    loaders["train"] = dataloaders.get_loader(batch_size=bs,
                                    data_name = 'imagenet',
                                    data_root = DATA_ROOT,
                                    num_workers = num_workers, 
                                    pin_memory = True)['train']

Building imagenet data loader with 32 workers
Building imagenet data loader with 32 workers


In [5]:
class ValCallback(Callback):
    
    
    def __init__(self, out_dir=None, out_prefix=None):
        self.out_dir = out_dir
        self.out_prefix = out_prefix
        self.predictions = defaultdict(lambda: [])
        self._keys_from_state = ["out_dir", "out_prefix"]

    def on_stage_start(self, state):
        for key in self._keys_from_state:
            value = getattr(state, key, None)
            if value is not None:
                setattr(self, key, value)
        # assert self.out_prefix is not None
        if self.out_dir is not None:
            self.out_prefix = str(self.out_dir) + "/" + str(self.out_prefix)
        if self.out_prefix is not None:
            os.makedirs(os.path.dirname(self.out_prefix), exist_ok=True)

    def on_loader_start(self, state):
        self.predictions = defaultdict(lambda: [])
        self.inputs = defaultdict(lambda: [])

    def on_batch_end(self, state):
        dct = state.output
        dct = {key: value.detach().cpu().numpy() for key, value in dct.items()}
        for key, value in dct.items():
            self.predictions[key].append(value)
            
        dct = state.input
        dct = {key: value.detach().cpu().numpy() for key, value in dct.items()}
        for key, value in dct.items():
            self.inputs[key].append(value)

    def on_loader_end(self, state):
        from catalyst.dl.metrics import accuracy
        self.predictions = {
            key: np.concatenate(value, axis=0)
            for key, value in self.predictions.items()
        }
        self.inputs = {
            key: np.concatenate(value, axis=0)
            for key, value in self.inputs.items()
        }
        y_true = torch.tensor(self.inputs['targets'])
        y_pred = torch.tensor(self.predictions['logits'])
        self.accuracy_score = torch.stack(accuracy(y_pred,y_true,topk=(1,5))).numpy().astype('float32')
        self.accuracy_score = np.squeeze(self.accuracy_score)

In [6]:
import sys
sys.path.append('../')

from model_utils import load_model
from torchvision.models import resnet34

# MODEL_NAME = 'vgg16_imagenet'
# MODEL_NAME = 'resnet50_imagenet'
# MODEL_NAME = 'resnet18_imagenet'
MODEL_NAME = 'resnet34_imagenet'

# MODEL_NAME = 'faster_rcnn_vgg16
# MODEL_NAME = 'faster_rcnn_resnet50'


# # Uncomment if MODEL_NAME = 'faster_rcnn_resnet50'
# sys.path.append('/workspace/home/jgusak/maxvol_objects/facebook_frcnn/')
# import maskrcnn_benchmark

# model = load_model(MODEL_NAME)

model = resnet34(pretrained = True)

In [7]:
lr = 0.002
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-6,lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

# model runner
runner = SupervisedRunner()

# model training
runner.train(
    model=model,
    criterion=criterion,
    callbacks=[
        AccuracyCallback(accuracy_args = [1,5]), # top (1,5) accuracy  on batch
#         ValCallback()
    ],
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    logdir='./logs3',
    num_epochs=5,
    verbose=True,
    valid_loader='valid'
    
)



0/5 * Epoch (valid): 100% 391/391 [01:06<00:00,  9.35it/s, _timers/_fps=503.193, accuracy01=46.250, accuracy05=86.250, loss=1.938]  
0/5 * Epoch (train):   9% 904/10010 [04:37<27:06,  5.60it/s, _timers/_fps=3444.328, accuracy01=18.750, accuracy05=35.938, loss=4.432]  

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0



0/5 * Epoch (train):  20% 2027/10010 [09:58<24:35,  5.41it/s, _timers/_fps=3294.899, accuracy01=21.875, accuracy05=49.219, loss=3.895]  

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0



0/5 * Epoch (train):  30% 3039/10010 [14:51<21:25,  5.42it/s, _timers/_fps=3140.716, accuracy01=32.812, accuracy05=57.812, loss=3.105]  

...

4/5 * Epoch (train):  84% 8440/10010 [43:32<08:54,  2.94it/s, _timers/_fps=307.607, accuracy01=56.250, accuracy05=78.125, loss=1.886]   

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0



4/5 * Epoch (train):  97% 9667/10010 [49:48<01:13,  4.67it/s, _timers/_fps=3200.289, accuracy01=64.062, accuracy05=88.281, loss=1.498]

/trinity/shared/opt/python-3.7.1/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:754: UserWarning:

Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0



4/5 * Epoch (train): 100% 10010/10010 [51:31<00:00,  3.24it/s, _timers/_fps=3887.158, accuracy01=53.333, accuracy05=80.000, loss=2.552]
[2019-06-23 02:41:09,435] 
4/5 * Epoch 4 (valid): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=8395.0922 | _timers/batch_time=0.1591 | _timers/data_time=0.1521 | _timers/model_time=0.0069 | accuracy01=60.4072 | accuracy05=83.0039 | loss=1.6686
4/5 * Epoch 4 (train): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=3006.1626 | _timers/batch_time=0.1636 | _timers/data_time=0.1548 | _timers/model_time=0.0087 | accuracy01=56.8314 | accuracy05=79.2064 | loss=1.8900
Top best models:
logs3/checkpoints//train.0.pth	1.0821
logs3/checkpoints//train.4.pth	1.6686
logs3/checkpoints//train.2.pth	1.9110


In [8]:
scores0 = run(model)

0/1 * Epoch (infer): 100% 391/391 [01:09<00:00,  6.08it/s, _timers/_fps=684.177, accuracy01=46.250, accuracy03=80.000, accuracy05=86.250]  


In [9]:
scores0

array([73.314   , 87.728004, 91.420006], dtype=float32)